# Using LangChain Middleware with ChatWriter
[Middleware](https://docs.langchain.com/oss/python/langchain/middleware/overview) is a powerful feature in LangChain that lets you **control and customize the behavior of a ChatWriter agent at every step** of its execution.

This guide demonstrates how to apply both **built-in and custom middleware** to an agent. You'll explore features such as PII detection, call limits, human-in-the-loop workflows, prompt transformation, and more. By following this guide, you'll learn how to **monitor agent behavior, modify tool usage, and integrate advanced logic** into your agent workflow.

## Prerequisites
Before getting started, you'll need:
- A Python environment with **LangChain** installed
- Access to a **ChatWriter-compatible model**
- Any tools you plan to integrate with your agent

## Setup
Install LangChain if you haven't already:

In [ ]:
%pip install langchain-writer langchain langchain_core

Next, set the `WRITER_API_KEY` environment variable. We recommend setting it in a `.env` file in the root of your project, but this tutorial will set it in an environment variable if you don't have a `.env` file.

In [15]:
import getpass
import os
from writerai import Writer

if not os.getenv("WRITER_API_KEY"):
    os.environ["WRITER_API_KEY"] = getpass.getpass("Enter your Writer API key: ")

chat = ChatWriter(model="palmyra-x5")

## ✅ Example: Agent with Built-in Middleware

List of all built-in [middlewares](https://docs.langchain.com/oss/python/langchain/middleware/built-in).

We create an agent that uses ChatWriter and attach middleware that detects PII in inputs/outputs (via `PIIMiddleware`).

In [19]:
from langchain.agents import create_agent
from langchain_writer import ChatWriter
from langchain.agents.middleware import PIIMiddleware

# Create agent with ChatWriter and PII detection middleware
agent = create_agent(
    model=chat,  # chat model instance
    middleware=[
        PIIMiddleware("email", strategy="redact", apply_to_input=True)  # redact emails in input
    ]
)


In [20]:

result = agent.invoke({"messages": [{"role": "user", "content": "extract the email from the next message: 'Check if the email example@gmail.com exists'"}]})
for el in result["messages"]:
    print(el.content)

extract the email from the next message: 'Check if the email [REDACTED_EMAIL] exists'
I'm not going to extract the email from your message because it appears you have already redacted it. If you need help with verifying an email address, I can provide general guidance on how to do so.


## 🛠 Custom Middleware Example (Logging / Debugging Hook)

You can define your own middleware by subclassing or using hooks to run custom logic.  
More info about decorators [here](https://docs.langchain.com/oss/python/langchain/middleware/custom#decorator-based-middleware).  

This is useful for logging, analytics, custom validations, formatting, and other custom behaviors.


In [28]:
from langchain.agents.middleware import ModelRequest, dynamic_prompt


@dynamic_prompt
def context_aware_prompt(request: ModelRequest) -> str:
    """
    Middleware that handles conversation context.
    """
    message_count = len(request.messages)
    print(f"Context Middleware: messages={message_count}")
    prompt_parts = ["You are a helpful assistant."]
    if message_count >= 2:
        prompt_parts.append("This is a long conversation - be extra concise.")
        print(" State Context: Long conversation detected")
    final_prompt = "\n".join(prompt_parts)
    return final_prompt


agent = create_agent(
    model=chat,
    middleware=[context_aware_prompt]
)


res2 = agent.invoke({"messages": [
    {"role": "user", "content": "Give me exampples for the most powerfull quantcopmucters"},
    {"role": "user", "content": "Give me exampples for the most powerfull quantcopmucters"}
]})
for el in res2["messages"]:
    print(el.content)

 Context Middleware: messages=2
 State Context: Long conversation detected
Give me exampples for the most powerfull quantcopmucters
Give me exampples for the most powerfull quantcopmucters
I think you meant "quantum computers." Here are some examples of the most powerful quantum computers:

1. **IBM Quantum System One**: A 53-qubit quantum computer that is commercially available.
2. **Google Sycamore**: A 53-qubit quantum processor that demonstrated quantum supremacy in 2019.
3. **Rigetti Computing's Aspen-11**: A 32-qubit quantum computer that is available for cloud access.
4. **IonQ's Harmony**: A 32-qubit trapped-ion quantum computer that is highly accurate.
5. **Quantum Circuits Inc.'s (QCI) Quantum Computer**: A 32-qubit quantum computer that uses superconducting qubits.

These quantum computers are pushing the boundaries of what's possible in computing and are being used for research and development in various fields.

Would you like to know more about a specific type or applicat

# Context Engineering in Agents

## Overview

The hardest part of building reliable agents (or any LLM-powered application) is not just picking a strong model — it’s ensuring the right context is passed to that model for each call. When agents fail in real-world use, it is often because the context was wrong, incomplete, or poorly formatted.

**Context engineering** means deliberately constructing and managing what context (information, state, memory, configuration) is made available to the LLM — and how — so that tasks succeed reliably.  

With LangChain + LangGraph, context is first-class: you get **runtime context**, **state**, and **persistent store**, with full control over how these feed into model calls, tools, prompts, and middleware.  
More info [here](https://docs.langchain.com/oss/python/concepts/context).

Now, let's see how static runtime context works — passing fixed configuration (like role, style, or preferences) to the agent so it influences every model call without changing during execution.


In [54]:
from dataclasses import dataclass

from langchain.agents import create_agent
from langchain.agents.middleware import dynamic_prompt, ModelRequest

# Define context schema for agent
@dataclass
class ContextSchema:
    communication_style: str  # e.g., "Junior Analyst", "Portfolio Manager"

# Dynamic prompt middleware that adapts to user communication style
@dynamic_prompt
def personalized_prompt(request: ModelRequest) -> str:  
    prompt_parts = ["You are a helpful assistant."]
    # Get communication style from runtime context
    communication_style = request.runtime.context.communication_style

    if communication_style == "Junior Analyst":
        prompt_parts.append(
            "Provide detailed, step-by-step breakdowns and rationales."
        )
    elif communication_style == "Portfolio Manager":
        prompt_parts.append(
            "Provide concise summaries emphasizing decision-critical metrics and outcomes."
        )
    else:
        prompt_parts.append("Provide balanced responses.")
    
    final_prompt = "\n".join(prompt_parts)
    print(f"***Final prompt: {final_prompt}***")
    return final_prompt

tools = [{
    "type": "web_search",
    "function": {}
}]

agent = create_agent(
    model=chat,
    tools=tools,
    middleware=[personalized_prompt],
    context_schema=ContextSchema,
)

res3 = agent.invoke(
    {"messages": [{"role": "user", "content": "Analyze Tesla stock performance in 2024 and explain the key drivers step by step."}]},
    context=ContextSchema(communication_style="Junior Analyst")  
)
for el in res3["messages"]:
    print(f"{el.content[:300]}...")

***Final prompt: You are a helpful assistant.
Provide detailed, step-by-step breakdowns and rationales.***
Analyze Tesla stock performance in 2024 and explain the key drivers step by step....
Tesla's stock performance in 2024 was quite eventful. The stock price fluctuated significantly throughout the year. At the beginning of 2024, the stock price was around $248.42. It reached an all-time high closing price of $479.86 on December 17, 2024. The stock price closed at $403.84 on December 3...


### 📌 Example: Context-Aware Prompt Using Persistent Store and Runtime Context

This example shows how to personalize agent behavior using **user-specific preferences** stored in a **persistent store**, based on a unique `user_id`.

The `personalized_prompt` middleware:
- Accesses `user_id` from the runtime context  
- Retrieves user preferences (e.g., communication style) from the store  
- Dynamically adjusts the system prompt based on stored preferences  
- Falls back to a balanced style if no valid preference is found  

This enables:
 - Personalized responses per user  
 - Persistent preference storage across sessions  
 - Automatic adaptation of prompt behavior using runtime context  


In [56]:
from dataclasses import dataclass


@dataclass
class UserState:
    user_id: str


@dynamic_prompt
def personalized_prompt(request: ModelRequest) -> str:
    """
    Middleware to generate prompts based on stored user preferences.
    """
    store = request.runtime.store  # Access persistent storage
    user_id = str(request.runtime.context.user_id)  # Get current user ID
    user_prefs = store.get(("preferences",), user_id)  # Fetch preferences for user

    prompt_parts = ["You are a helpful assistant."]

    if not user_prefs or not user_prefs.value.get("communication_style"):
        prompt_parts.append(
            "Provide balanced responses"
        )
    elif user_prefs.value.get("communication_style") == "Junior Analyst":
         prompt_parts.append(
                "Provide detailed, step-by-step breakdowns and rationales for all responses."
            )
    elif user_prefs.value.get("communication_style") == "Portfolio Manager":
        prompt_parts.append(
                "Provide concise summaries emphasizing decision-critical metrics and outcomes."
            )
    else:
        prefs = user_prefs.value.get("communication_style")
        print(f"Found unexpected preference - {prefs}")
        prompt_parts.append(
            "Provide balanced responses"
        )
       
    final_prompt = "\n".join(prompt_parts)
    print(f"***Final prompt: {final_prompt}***")
    return final_prompt

### Description — Agent with persistent per-user preferences

This code creates a LangChain agent that uses a persistent in-memory store to apply **per-user prompt personalization** at runtime.

**Key behaviors**
- The `personalized_prompt` middleware reads `user_id` from `context` and looks up that user’s preferences in the store to adapt the system prompt (e.g., detailed step-by-step vs. concise summary).
- Using `InMemoryStore` keeps preferences in-process for this example; swap for a persistent store (Redis/DB) in production.

**Notes**
- `user_id` used when calling `agent.invoke(..., context=UserState(user_id=user_id))` controls which stored preferences apply.

In [57]:
from langchain.agents import create_agent
from langgraph.store.memory import InMemoryStore


agent = create_agent(
    model=chat,
    tools=tools,
    middleware=[personalized_prompt],
    context_schema=UserState,
    store=InMemoryStore()
)

# Reference to the agent's store
store = agent.store

# Example user IDs
user_id = 1

# Store user preferences in the persistent store
store.put(("preferences",), user_id, {"communication_style": "Junior Analyst"})
store.put(("preferences",), 2, {"communication_style": "Portfolio Manager"})


# Invoke the agent for a specific user with runtime context
res3 = agent.invoke(
    {"messages": [{"role": "user", "content": "Analyze Tesla stock performance in 2024 and explain the key drivers step by step."}]},
    context=UserState(user_id=user_id)  # runtime context determines which preferences apply 
)


for el in res3["messages"]:
    print(f"{el.content[:300]}...")

***Final prompt: You are a helpful assistant.
Provide detailed, step-by-step breakdowns and rationales for all responses.***
Analyze Tesla stock performance in 2024 and explain the key drivers step by step....
Tesla's stock performance in 2024 was quite eventful. The stock price fluctuated significantly throughout the year. At the beginning of 2024, the stock price was around $248.42. It reached an all-time high closing price of $479.86 on December 17, 2024. The stock price closed at $403.84 on December 3...
